In [1]:
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing, model_selection, neighbors
# from sklearn.svm import LinearSVC, SVC
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import seaborn as sns
# from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from mlxtend.preprocessing import shuffle_arrays_unison
from sklearn.metrics import accuracy_score

In [3]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [4]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Age","Fare"]

In [6]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
df_train['Age'].fillna(df_train['Age'].mean(),inplace=True)
df_test['Age'].fillna(df_test['Age'].mean(),inplace=True)

In [10]:
df_train.drop(columns=['Cabin','Ticket',"Name","PassengerId"],axis=1,inplace=True)
df_test.drop(columns=['Cabin','Ticket',"Name","PassengerId"],axis=1,inplace=True)


In [11]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [12]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,108.9000,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [13]:
df_train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [14]:
# mapp={"S":0,"1":2,"Q":3}
# df_test['Embarked']=df_test['Embarked'].map(mapp)
# df_train['Embarked']=df_train['Embarked'].map(mapp)

In [15]:
mapp2={"male":1,"female":0}
df_test['Sex']=df_test['Sex'].map(mapp2)
df_train['Sex']=df_train['Sex'].map(mapp2)

In [16]:
df_train["Parch"].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [17]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.corr(),annot=True,cmap="RdYlGn")
# plt.show()

In [18]:
# sns.boxplot(df_train.iloc[:,3])

In [19]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.000000,1,0,7.2500,S
1,1,1,0,38.000000,1,0,71.2833,C
2,1,3,0,26.000000,0,0,7.9250,S
3,1,1,0,35.000000,1,0,53.1000,S
4,0,3,1,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,S
887,1,1,0,19.000000,0,0,30.0000,S
888,0,3,0,29.699118,1,2,23.4500,S
889,1,1,1,26.000000,0,0,30.0000,C


In [20]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.50000,0,0,7.8292,Q
1,3,0,47.00000,1,0,7.0000,S
2,2,1,62.00000,0,0,9.6875,Q
3,3,1,27.00000,0,0,8.6625,S
4,3,0,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,1,30.27259,0,0,8.0500,S
414,1,0,39.00000,0,0,108.9000,C
415,3,1,38.50000,0,0,7.2500,S
416,3,1,30.27259,0,0,8.0500,S


In [21]:
df_train.drop("Embarked",axis=1,inplace=True)

In [22]:
# dummies_Embarked=pd.DataFrame(pd.get_dummies(df_train['Embarked']))
# df_train.drop("Embarked",inplace=True,axis=1)
# df_train=pd.concat([dummies_Embarked,df_train],axis=1)

In [23]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.000000,1,0,7.2500
1,1,1,0,38.000000,1,0,71.2833
2,1,3,0,26.000000,0,0,7.9250
3,1,1,0,35.000000,1,0,53.1000
4,0,3,1,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000
887,1,1,0,19.000000,0,0,30.0000
888,0,3,0,29.699118,1,2,23.4500
889,1,1,1,26.000000,0,0,30.0000


In [24]:
df_test.drop("Embarked",axis=1,inplace=True)

In [25]:
# dummies_Embarked_test=pd.DataFrame(pd.get_dummies(df_test['Embarked']))
# df_test.drop("Embarked",inplace=True,axis=1)
# df_test=pd.concat([dummies_Embarked_test,df_test],axis=1)

In [26]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,34.50000,0,0,7.8292
1,3,0,47.00000,1,0,7.0000
2,2,1,62.00000,0,0,9.6875
3,3,1,27.00000,0,0,8.6625
4,3,0,22.00000,1,1,12.2875
...,...,...,...,...,...,...
413,3,1,30.27259,0,0,8.0500
414,1,0,39.00000,0,0,108.9000
415,3,1,38.50000,0,0,7.2500
416,3,1,30.27259,0,0,8.0500


In [27]:
df_train.dropna(inplace=True)

In [28]:
df_test.dropna(inplace=True)

In [29]:
df_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,1.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [30]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.0,1.0,1.0,0.271174,0.125,0.000000,0.014151
1,1.0,0.0,0.0,0.472229,0.125,0.000000,0.139136
2,1.0,1.0,0.0,0.321438,0.000,0.000000,0.015469
3,1.0,0.0,0.0,0.434531,0.125,0.000000,0.103644
4,0.0,1.0,1.0,0.434531,0.000,0.000000,0.015713
...,...,...,...,...,...,...,...
886,0.0,0.5,1.0,0.334004,0.000,0.000000,0.025374
887,1.0,0.0,0.0,0.233476,0.000,0.000000,0.058556
888,0.0,1.0,0.0,0.367921,0.125,0.333333,0.045771
889,1.0,0.0,1.0,0.321438,0.000,0.000000,0.058556


In [31]:
scaler=scaler=MinMaxScaler()
df_test=pd.DataFrame(scaler.fit_transform(df_test),columns=df_test.columns,)
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1.0,1.0,0.452723,0.000,0.000000,0.015282
1,1.0,0.0,0.617566,0.125,0.000000,0.013663
2,0.5,1.0,0.815377,0.000,0.000000,0.018909
3,1.0,1.0,0.353818,0.000,0.000000,0.016908
4,1.0,0.0,0.287881,0.125,0.111111,0.023984
...,...,...,...,...,...,...
412,1.0,1.0,0.396975,0.000,0.000000,0.015713
413,0.0,0.0,0.512066,0.000,0.000000,0.212559
414,1.0,1.0,0.505473,0.000,0.000000,0.014151
415,1.0,1.0,0.396975,0.000,0.000000,0.015713


In [32]:
X=np.array(df_train.drop("Survived",axis=1))
Y=np.array(df_train['Survived']).astype('int32')

In [35]:
print(Y.dtype,X.dtype)

int32 float64


In [36]:
X,Y=shuffle_arrays_unison(arrays=[X,Y],random_seed=3)

In [37]:

X_train,X_test,Y_train,Y_test=model_selection.train_test_split(X,Y,test_size=0.2,random_state=42)

In [50]:

class DecisionTree:
    
    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None
        
    def _leaf_value(self, y):
    # return the most common class label in y
        counts = np.bincount(y)
        return np.argmax(counts)

        
    def fit(self, X, y):
        self.tree = self._grow_tree(X, y)
        
    def predict(self, X):
        return [self._predict_tree(x, self.tree) for x in X]
    
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Stop conditions
        if depth == self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._leaf_value(y)
            return {'leaf': True, 'value': leaf_value}
        
        # Splitting
        feature_idxs = np.random.choice(n_features, int(np.sqrt(n_features)), replace=False)
        best_feature, best_threshold = self._best_criteria(X, y, feature_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        
        # Check minimum sample requirement
        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value = self._leaf_value(y)
            return {'leaf': True, 'value': leaf_value}
        
        # Grow subtrees
        left_tree = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right_tree = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        
        return {'leaf': False, 'feature_idx': best_feature, 'threshold': best_threshold,
                'left': left_tree, 'right': right_tree}
        
    def _best_criteria(self, X, y, feature_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None
        for feature_idx in feature_idxs:
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = threshold
        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, split_threshold):
        parent_entropy = self._entropy(y)
        
        # Generate split indices
        left_idxs, right_idxs = self._split(X_column, split_threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # Calculate the weighted average of child entropies
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        entropy_l, entropy_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * entropy_l + (n_r/n) * entropy_r
        
        # Return the information gain
        return parent_entropy - child_entropy
    
    def _split(self, X_column, split_threshold):
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column > split_threshold).flatten()
        return left_idxs, right_idxs
    
    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))
        return entropy
    
    def _predict_tree(self, x, tree):
        if tree['leaf']:
            return tree['value']
        else:
            feature_value = x[tree['feature_idx']]
            if feature_value <= tree['threshold']:
                return self._predict_tree(x, tree['left'])
            else:
                return self._predict_tree(x, tree['right'])


 

        

class RandomForest:
    def __init__(self, classifier, num_trees, min_features):
        self.classifier = classifier
        self.num_trees = num_trees
        self.min_features = min_features
        self.trees = []

    def fit(self, X, y):
        for i in range(self.num_trees):
            # Sampling with Replacement
            idx = np.random.choice(X.shape[0], X.shape[0], replace=True)
            X_subset = X[idx]
            y_subset = y[idx]

            # Selecting Features
            num_features = np.random.randint(self.min_features, X.shape[1])
            feature_indices = np.random.choice(X.shape[1], num_features, replace=False)

            X_subset = X_subset[:, feature_indices]

            # Creating decision tree
            tree = DecisionTree(self.classifier)
            tree.fit(X_subset, y_subset)

            self.trees.append((tree, feature_indices))

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.trees)))
        for i, (tree, feature_indices) in enumerate(self.trees):
            X_subset = X[:, feature_indices]
            predictions[:, i] = tree.predict(X_subset)

        # Voting
        final_predictions = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
#             final_predictions[i] = np.argmax(np.bincount(predictions[i, :]))
            
            final_predictions[i] = np.argmax(np.bincount(predictions[i, :].astype(int)))





        return final_predictions
    



In [51]:

from sklearn.metrics import accuracy_score

np_array_of_test=np.array(df_test)
# Create an instance of the DecisionTree class
    # tree = DecisionTree(max_depth=3, min_samples_split=5, min_samples_leaf=3)
    
tree = DecisionTree(criterion='gini',max_depth=3, min_samples_split=5, min_samples_leaf=3)

# Instantiate a RandomForest instance with the DecisionTree instance as the classifier
rf = RandomForest(tree, num_trees=10, min_features=2)
    
    
    # Fit the model on the training data
rf.fit(X_train, Y_train)


    # Make predictions on the testing data
y_pred= rf.predict(X_test)

#      Calculate the accuracy of the model
acc = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
print()

Accuracy: 0.799



In [52]:
# Predict of tes
prediction=rf.predict(np_array_of_test)


In [53]:
prediction

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 0.